In [40]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import csv

In [41]:
HEADERS = ({
    'User-Agent':'Mozilla/5.0 (X11; Linux x86_64)AppleWebKit/537.36 (KHTML, like Gecko)Chrome/44.0.2403.157 Safari/537.36',
    'Accept-Language': 'en-US, en;q=0.5'
})

In [42]:
# page_number = 0
# status = 200
# i=0
# while True:
#     i+=1
#     url = f'https://www.themoviedb.org/movie?page={i}'
#     response = requests.get(url,headers=HEADERS)
#     print(i,end=' ')
#     if(response.status_code==200):
#         page_number+=1
#     else:
#         break
    
# print('total number of pages are :',page_number)


## commented this code because it was taking too long to run
## and i have to run whole file again and again after opening this

NOTE : Total number of pages are 500

In [58]:
movie_data_list = list()

In [59]:
def get_budget_and_revenue(page_soup):
    facts = page_soup.find('section',class_='facts left_column')
    if facts is not None:
        title,movie_budget = facts.find_all('bdi')[2].parent.parent.text.split()
        movie_budget = movie_budget.replace('$','')
        title,movie_revenue = facts.find_all('bdi')[3].parent.parent.text.split()
        movie_revenue = movie_revenue.replace('$','')
        movie_budget = float(movie_budget.replace(',',''))/(10**6) if movie_budget != '-' else np.nan
        movie_revenue = float(movie_revenue.replace(',',''))/(10**6) if movie_revenue != '-' else np.nan
        return movie_budget,movie_revenue
    return np.nan,np.nan

In [60]:
def get_title(page_soup):
    movie_title = page_soup.find('h2')
    return movie_title.find('a').text if movie_title is not None else 'unknown'

In [61]:
def get_user_score(page_soup):
    percentage = page_soup.find('div',class_='percent')
    if percentage is not None:
        percentage_string = percentage.find('span')['class'][1]
        index = percentage_string.index('r')
        user_score = percentage_string[index+1:] 
        try:
            user_score = eval(user_score)
            return user_score
        except:
            return np.nan
    return np.nan

In [62]:
def get_genres(page_soup):
    genres_list = 'unknown'
    genres = page_soup.find('span',class_='genres')
    return ','.join([genre.text for genre in genres.find_all('a')]) if genres is not None else 'unknown'

In [63]:
def get_release_date(page_soup):
    release_date_tag = page_soup.find('span',class_='release')
    if release_date_tag is not None:
        release_date,temp = release_date_tag.text.replace('\n','').split()
        year = pd.to_datetime(release_date).year
        month = pd.to_datetime(release_date).month
        day = pd.to_datetime(release_date).day
        weekday = pd.to_datetime(release_date).weekday()
        return year,month,day,weekday
    return np.nan,np.nan,np.nan,np.nan

In [64]:
def get_director_name(page_soup):
    director = page_soup.find('li',class_='profile')
    return director.find('a').text if director is not None else 'unknown'

In [65]:
def get_run_time(page_soup):
    run_time = page_soup.find('span',class_='runtime')
    if run_time is not None:
        time_string = run_time.text.replace('\n','').replace(' ','').replace('h',':').replace('m','')
        hours,minuts = time_string.split(':') if ':' in time_string else ('',time_string)
        hours = eval(hours) if hours != '' else 0
        minuts = eval(minuts) if minuts != '' else 0
        total_duration = (hours*60)+minuts
        return total_duration
    else:
        return np.nan

In [66]:
def get_content_score(page_soup):
    content_score_tag = page_soup.find('div',class_='content_score')
    return eval(content_score_tag.find('p').text) if content_score_tag is not None else np.nan

In [67]:
def get_movie_data(response):
    
    page_soup = BeautifulSoup(response.text,'html.parser')
    
    content_score = get_content_score(page_soup)
    
    budget,revenue = get_budget_and_revenue(page_soup)
    movie_title = get_title(page_soup)
    user_score = get_user_score(page_soup)
    genres = get_genres(page_soup)
    year,month,day,weekday = get_release_date(page_soup)
    director = get_director_name(page_soup)
    run_time = get_run_time(page_soup)
    return {
        'budget(in million)':budget,
        'revenue(in million)':revenue,
        'content_score':content_score,
        'user_score':user_score,
        'genres':genres,
        'director':director,
        'release_year':year,
        'release_month':month,
        'release_day':day,
        'release_weekday':weekday,
        'run_time (in minuts)':run_time,
        'title':movie_title
    }

In [68]:
def get_movies(movie_data):
    for movie in movie_data:    
        movie_page_url = f"https://www.themoviedb.org{movie.find('a')['href']}"
        print(movie_page_url)
        movie_page_response = requests.get(movie_page_url,headers=HEADERS)
        movie_info = get_movie_data(movie_page_response)
        movie_info['url'] = movie_page_url
        movie_data_list.append(movie_info)
        

In [69]:
def get_data(starting=1,ending=10):
    for i in range(starting,ending+1):
        print(f'page number {i}')
        url = f'https://www.themoviedb.org/movie?page={i}'
        response = requests.get(url,headers=HEADERS)
        page= BeautifulSoup(response.text, 'html.parser')
        movie_data = page.find_all('div',class_='card style_1')
        get_movies(movie_data = movie_data)
        print('\n')
    
    print("DONE")

In [70]:
get_data(starting=1,ending=100)

page number 1
https://www.themoviedb.org/movie/566525
https://www.themoviedb.org/movie/370172
https://www.themoviedb.org/movie/580489
https://www.themoviedb.org/movie/522402
https://www.themoviedb.org/movie/763164
https://www.themoviedb.org/movie/744275
https://www.themoviedb.org/movie/524434
https://www.themoviedb.org/movie/885110
https://www.themoviedb.org/movie/438631
https://www.themoviedb.org/movie/796499
https://www.themoviedb.org/movie/618162
https://www.themoviedb.org/movie/550988
https://www.themoviedb.org/movie/512195
https://www.themoviedb.org/movie/585245
https://www.themoviedb.org/movie/775943
https://www.themoviedb.org/movie/843241
https://www.themoviedb.org/movie/714968
https://www.themoviedb.org/movie/335983
https://www.themoviedb.org/movie/672582
https://www.themoviedb.org/movie/574060


page number 2
https://www.themoviedb.org/movie/568620
https://www.themoviedb.org/movie/871964
https://www.themoviedb.org/movie/854442
https://www.themoviedb.org/movie/639721
https://ww

https://www.themoviedb.org/movie/590706
https://www.themoviedb.org/movie/475557
https://www.themoviedb.org/movie/198663
https://www.themoviedb.org/movie/324786
https://www.themoviedb.org/movie/773660
https://www.themoviedb.org/movie/567189
https://www.themoviedb.org/movie/24021
https://www.themoviedb.org/movie/613583
https://www.themoviedb.org/movie/871875
https://www.themoviedb.org/movie/512025
https://www.themoviedb.org/movie/784500
https://www.themoviedb.org/movie/283566
https://www.themoviedb.org/movie/882812
https://www.themoviedb.org/movie/809314
https://www.themoviedb.org/movie/877183
https://www.themoviedb.org/movie/4564
https://www.themoviedb.org/movie/571891
https://www.themoviedb.org/movie/410113


page number 12
https://www.themoviedb.org/movie/740925
https://www.themoviedb.org/movie/347626
https://www.themoviedb.org/movie/741074
https://www.themoviedb.org/movie/10138
https://www.themoviedb.org/movie/719164
https://www.themoviedb.org/movie/474350
https://www.themoviedb.org/

https://www.themoviedb.org/movie/366170
https://www.themoviedb.org/movie/8355
https://www.themoviedb.org/movie/381749
https://www.themoviedb.org/movie/534
https://www.themoviedb.org/movie/736069
https://www.themoviedb.org/movie/14836
https://www.themoviedb.org/movie/587996
https://www.themoviedb.org/movie/49529
https://www.themoviedb.org/movie/348893
https://www.themoviedb.org/movie/119450
https://www.themoviedb.org/movie/341174
https://www.themoviedb.org/movie/259693
https://www.themoviedb.org/movie/126963
https://www.themoviedb.org/movie/589761
https://www.themoviedb.org/movie/793723


page number 22
https://www.themoviedb.org/movie/778552
https://www.themoviedb.org/movie/788905
https://www.themoviedb.org/movie/776909
https://www.themoviedb.org/movie/2062
https://www.themoviedb.org/movie/76338
https://www.themoviedb.org/movie/120
https://www.themoviedb.org/movie/38365
https://www.themoviedb.org/movie/9637
https://www.themoviedb.org/movie/157336
https://www.themoviedb.org/movie/42194


https://www.themoviedb.org/movie/726739
https://www.themoviedb.org/movie/408
https://www.themoviedb.org/movie/49026
https://www.themoviedb.org/movie/4258
https://www.themoviedb.org/movie/542178
https://www.themoviedb.org/movie/20222
https://www.themoviedb.org/movie/576393
https://www.themoviedb.org/movie/514439
https://www.themoviedb.org/movie/136797
https://www.themoviedb.org/movie/560050
https://www.themoviedb.org/movie/23047


page number 32
https://www.themoviedb.org/movie/265712
https://www.themoviedb.org/movie/168259
https://www.themoviedb.org/movie/16996
https://www.themoviedb.org/movie/278154
https://www.themoviedb.org/movie/817451
https://www.themoviedb.org/movie/5255
https://www.themoviedb.org/movie/364689
https://www.themoviedb.org/movie/9502
https://www.themoviedb.org/movie/52520
https://www.themoviedb.org/movie/260513
https://www.themoviedb.org/movie/39101
https://www.themoviedb.org/movie/159824
https://www.themoviedb.org/movie/950
https://www.themoviedb.org/movie/767504
h

https://www.themoviedb.org/movie/381288
https://www.themoviedb.org/movie/343668
https://www.themoviedb.org/movie/648990
https://www.themoviedb.org/movie/874965
https://www.themoviedb.org/movie/37786
https://www.themoviedb.org/movie/400928
https://www.themoviedb.org/movie/621876


page number 42
https://www.themoviedb.org/movie/638134
https://www.themoviedb.org/movie/614917
https://www.themoviedb.org/movie/157350
https://www.themoviedb.org/movie/51608
https://www.themoviedb.org/movie/342470
https://www.themoviedb.org/movie/17979
https://www.themoviedb.org/movie/484718
https://www.themoviedb.org/movie/245891
https://www.themoviedb.org/movie/4523
https://www.themoviedb.org/movie/278
https://www.themoviedb.org/movie/888917
https://www.themoviedb.org/movie/413594
https://www.themoviedb.org/movie/81774
https://www.themoviedb.org/movie/817566
https://www.themoviedb.org/movie/347548
https://www.themoviedb.org/movie/6499
https://www.themoviedb.org/movie/620
https://www.themoviedb.org/movie/1818

https://www.themoviedb.org/movie/600583
https://www.themoviedb.org/movie/250546
https://www.themoviedb.org/movie/454699


page number 52
https://www.themoviedb.org/movie/20941
https://www.themoviedb.org/movie/576004
https://www.themoviedb.org/movie/2758
https://www.themoviedb.org/movie/2770
https://www.themoviedb.org/movie/200085
https://www.themoviedb.org/movie/17610
https://www.themoviedb.org/movie/523536
https://www.themoviedb.org/movie/796849
https://www.themoviedb.org/movie/19913
https://www.themoviedb.org/movie/24122
https://www.themoviedb.org/movie/10184
https://www.themoviedb.org/movie/124905
https://www.themoviedb.org/movie/147441
https://www.themoviedb.org/movie/594718
https://www.themoviedb.org/movie/76163
https://www.themoviedb.org/movie/16985
https://www.themoviedb.org/movie/81796
https://www.themoviedb.org/movie/553839
https://www.themoviedb.org/movie/10527
https://www.themoviedb.org/movie/136795


page number 53
https://www.themoviedb.org/movie/802
https://www.themoviedb

https://www.themoviedb.org/movie/311324
https://www.themoviedb.org/movie/16523
https://www.themoviedb.org/movie/299687
https://www.themoviedb.org/movie/242582
https://www.themoviedb.org/movie/60747
https://www.themoviedb.org/movie/656561
https://www.themoviedb.org/movie/454286
https://www.themoviedb.org/movie/54138
https://www.themoviedb.org/movie/752455
https://www.themoviedb.org/movie/256040
https://www.themoviedb.org/movie/12556
https://www.themoviedb.org/movie/671266
https://www.themoviedb.org/movie/462883
https://www.themoviedb.org/movie/76589
https://www.themoviedb.org/movie/10693
https://www.themoviedb.org/movie/16577
https://www.themoviedb.org/movie/84226
https://www.themoviedb.org/movie/845222
https://www.themoviedb.org/movie/618344


page number 63
https://www.themoviedb.org/movie/195589
https://www.themoviedb.org/movie/14926
https://www.themoviedb.org/movie/761898
https://www.themoviedb.org/movie/889753
https://www.themoviedb.org/movie/559581
https://www.themoviedb.org/movie

https://www.themoviedb.org/movie/774810
https://www.themoviedb.org/movie/590223
https://www.themoviedb.org/movie/127533
https://www.themoviedb.org/movie/41733
https://www.themoviedb.org/movie/410685
https://www.themoviedb.org/movie/213927
https://www.themoviedb.org/movie/136835
https://www.themoviedb.org/movie/11351
https://www.themoviedb.org/movie/45612
https://www.themoviedb.org/movie/10867
https://www.themoviedb.org/movie/55779
https://www.themoviedb.org/movie/146368
https://www.themoviedb.org/movie/619432
https://www.themoviedb.org/movie/379291


page number 73
https://www.themoviedb.org/movie/14128
https://www.themoviedb.org/movie/11036
https://www.themoviedb.org/movie/297270
https://www.themoviedb.org/movie/101299
https://www.themoviedb.org/movie/192141
https://www.themoviedb.org/movie/288985
https://www.themoviedb.org/movie/595149
https://www.themoviedb.org/movie/11631
https://www.themoviedb.org/movie/391757
https://www.themoviedb.org/movie/400136
https://www.themoviedb.org/movi

https://www.themoviedb.org/movie/340382
https://www.themoviedb.org/movie/266285
https://www.themoviedb.org/movie/175112
https://www.themoviedb.org/movie/76617
https://www.themoviedb.org/movie/820654
https://www.themoviedb.org/movie/13700
https://www.themoviedb.org/movie/615665
https://www.themoviedb.org/movie/579051
https://www.themoviedb.org/movie/55301
https://www.themoviedb.org/movie/63311


page number 83
https://www.themoviedb.org/movie/716258
https://www.themoviedb.org/movie/373879
https://www.themoviedb.org/movie/41446
https://www.themoviedb.org/movie/598
https://www.themoviedb.org/movie/351460
https://www.themoviedb.org/movie/638507
https://www.themoviedb.org/movie/64694
https://www.themoviedb.org/movie/238713
https://www.themoviedb.org/movie/22970
https://www.themoviedb.org/movie/44282
https://www.themoviedb.org/movie/6114
https://www.themoviedb.org/movie/180299
https://www.themoviedb.org/movie/726208
https://www.themoviedb.org/movie/201088
https://www.themoviedb.org/movie/101

https://www.themoviedb.org/movie/1366
https://www.themoviedb.org/movie/171424
https://www.themoviedb.org/movie/733491
https://www.themoviedb.org/movie/16871
https://www.themoviedb.org/movie/4951


page number 93
https://www.themoviedb.org/movie/25475
https://www.themoviedb.org/movie/109520
https://www.themoviedb.org/movie/585381
https://www.themoviedb.org/movie/82703
https://www.themoviedb.org/movie/26338
https://www.themoviedb.org/movie/216282
https://www.themoviedb.org/movie/551804
https://www.themoviedb.org/movie/28609
https://www.themoviedb.org/movie/1091
https://www.themoviedb.org/movie/7485
https://www.themoviedb.org/movie/152601
https://www.themoviedb.org/movie/954
https://www.themoviedb.org/movie/274167
https://www.themoviedb.org/movie/18937
https://www.themoviedb.org/movie/218
https://www.themoviedb.org/movie/252838
https://www.themoviedb.org/movie/18405
https://www.themoviedb.org/movie/14574
https://www.themoviedb.org/movie/694256
https://www.themoviedb.org/movie/582596


pag

In [71]:
get_data(starting=101,ending=200)

page number 101
https://www.themoviedb.org/movie/10096
https://www.themoviedb.org/movie/4348
https://www.themoviedb.org/movie/257211
https://www.themoviedb.org/movie/41513
https://www.themoviedb.org/movie/748224
https://www.themoviedb.org/movie/283378
https://www.themoviedb.org/movie/799497
https://www.themoviedb.org/movie/778730
https://www.themoviedb.org/movie/325358
https://www.themoviedb.org/movie/109414
https://www.themoviedb.org/movie/426543
https://www.themoviedb.org/movie/248509
https://www.themoviedb.org/movie/602
https://www.themoviedb.org/movie/549787
https://www.themoviedb.org/movie/18
https://www.themoviedb.org/movie/608
https://www.themoviedb.org/movie/866343
https://www.themoviedb.org/movie/323272
https://www.themoviedb.org/movie/59118
https://www.themoviedb.org/movie/672741


page number 102
https://www.themoviedb.org/movie/375588
https://www.themoviedb.org/movie/581734
https://www.themoviedb.org/movie/483455
https://www.themoviedb.org/movie/376660
https://www.themovied

https://www.themoviedb.org/movie/609242
https://www.themoviedb.org/movie/796822
https://www.themoviedb.org/movie/1690
https://www.themoviedb.org/movie/583903
https://www.themoviedb.org/movie/15969
https://www.themoviedb.org/movie/699102
https://www.themoviedb.org/movie/514593
https://www.themoviedb.org/movie/10515
https://www.themoviedb.org/movie/646
https://www.themoviedb.org/movie/39451
https://www.themoviedb.org/movie/2048
https://www.themoviedb.org/movie/1585
https://www.themoviedb.org/movie/438799
https://www.themoviedb.org/movie/418437
https://www.themoviedb.org/movie/112454
https://www.themoviedb.org/movie/372758


page number 112
https://www.themoviedb.org/movie/50646
https://www.themoviedb.org/movie/653670
https://www.themoviedb.org/movie/447399
https://www.themoviedb.org/movie/361292
https://www.themoviedb.org/movie/13183
https://www.themoviedb.org/movie/164251
https://www.themoviedb.org/movie/16538
https://www.themoviedb.org/movie/74078
https://www.themoviedb.org/movie/78821

https://www.themoviedb.org/movie/745
https://www.themoviedb.org/movie/340666
https://www.themoviedb.org/movie/204082
https://www.themoviedb.org/movie/249164
https://www.themoviedb.org/movie/423
https://www.themoviedb.org/movie/9312
https://www.themoviedb.org/movie/395990
https://www.themoviedb.org/movie/500713
https://www.themoviedb.org/movie/763788
https://www.themoviedb.org/movie/318846
https://www.themoviedb.org/movie/467433


page number 122
https://www.themoviedb.org/movie/158852
https://www.themoviedb.org/movie/79113
https://www.themoviedb.org/movie/785976
https://www.themoviedb.org/movie/243683
https://www.themoviedb.org/movie/491480
https://www.themoviedb.org/movie/8095
https://www.themoviedb.org/movie/39691
https://www.themoviedb.org/movie/390054
https://www.themoviedb.org/movie/820752
https://www.themoviedb.org/movie/22824
https://www.themoviedb.org/movie/10014
https://www.themoviedb.org/movie/36669
https://www.themoviedb.org/movie/821664
https://www.themoviedb.org/movie/6003

https://www.themoviedb.org/movie/586097
https://www.themoviedb.org/movie/78192
https://www.themoviedb.org/movie/10634
https://www.themoviedb.org/movie/343674
https://www.themoviedb.org/movie/8656
https://www.themoviedb.org/movie/472035
https://www.themoviedb.org/movie/256961


page number 132
https://www.themoviedb.org/movie/298250
https://www.themoviedb.org/movie/4858
https://www.themoviedb.org/movie/1894
https://www.themoviedb.org/movie/513268
https://www.themoviedb.org/movie/9969
https://www.themoviedb.org/movie/821936
https://www.themoviedb.org/movie/489245
https://www.themoviedb.org/movie/88278
https://www.themoviedb.org/movie/14863
https://www.themoviedb.org/movie/9823
https://www.themoviedb.org/movie/9763
https://www.themoviedb.org/movie/441701
https://www.themoviedb.org/movie/452970
https://www.themoviedb.org/movie/82654
https://www.themoviedb.org/movie/17101
https://www.themoviedb.org/movie/625169
https://www.themoviedb.org/movie/357786
https://www.themoviedb.org/movie/38356
h

https://www.themoviedb.org/movie/513223
https://www.themoviedb.org/movie/10045
https://www.themoviedb.org/movie/211387


page number 142
https://www.themoviedb.org/movie/714
https://www.themoviedb.org/movie/142
https://www.themoviedb.org/movie/56903
https://www.themoviedb.org/movie/551238
https://www.themoviedb.org/movie/226486
https://www.themoviedb.org/movie/595813
https://www.themoviedb.org/movie/484886
https://www.themoviedb.org/movie/74
https://www.themoviedb.org/movie/11459
https://www.themoviedb.org/movie/10131
https://www.themoviedb.org/movie/599399
https://www.themoviedb.org/movie/740985
https://www.themoviedb.org/movie/1422
https://www.themoviedb.org/movie/600
https://www.themoviedb.org/movie/776527
https://www.themoviedb.org/movie/500
https://www.themoviedb.org/movie/421313
https://www.themoviedb.org/movie/9874
https://www.themoviedb.org/movie/360784
https://www.themoviedb.org/movie/643550


page number 143
https://www.themoviedb.org/movie/323372
https://www.themoviedb.org/m

https://www.themoviedb.org/movie/537055
https://www.themoviedb.org/movie/2252
https://www.themoviedb.org/movie/218836
https://www.themoviedb.org/movie/839
https://www.themoviedb.org/movie/408266
https://www.themoviedb.org/movie/251519
https://www.themoviedb.org/movie/48171
https://www.themoviedb.org/movie/175574
https://www.themoviedb.org/movie/63
https://www.themoviedb.org/movie/629
https://www.themoviedb.org/movie/313943
https://www.themoviedb.org/movie/36648
https://www.themoviedb.org/movie/389015
https://www.themoviedb.org/movie/107811
https://www.themoviedb.org/movie/2135
https://www.themoviedb.org/movie/410718
https://www.themoviedb.org/movie/11675
https://www.themoviedb.org/movie/535292


page number 153
https://www.themoviedb.org/movie/11587
https://www.themoviedb.org/movie/9471
https://www.themoviedb.org/movie/555748
https://www.themoviedb.org/movie/99005
https://www.themoviedb.org/movie/44826
https://www.themoviedb.org/movie/9711
https://www.themoviedb.org/movie/11199
https:/

https://www.themoviedb.org/movie/632304
https://www.themoviedb.org/movie/594
https://www.themoviedb.org/movie/432985
https://www.themoviedb.org/movie/168672
https://www.themoviedb.org/movie/823461
https://www.themoviedb.org/movie/463821
https://www.themoviedb.org/movie/50388
https://www.themoviedb.org/movie/698201
https://www.themoviedb.org/movie/449664
https://www.themoviedb.org/movie/293768
https://www.themoviedb.org/movie/205775
https://www.themoviedb.org/movie/420814
https://www.themoviedb.org/movie/663459


page number 163
https://www.themoviedb.org/movie/454458
https://www.themoviedb.org/movie/331313
https://www.themoviedb.org/movie/309809
https://www.themoviedb.org/movie/17165
https://www.themoviedb.org/movie/316322
https://www.themoviedb.org/movie/138697
https://www.themoviedb.org/movie/291549
https://www.themoviedb.org/movie/337170
https://www.themoviedb.org/movie/8916
https://www.themoviedb.org/movie/438747
https://www.themoviedb.org/movie/293767
https://www.themoviedb.org/mo

https://www.themoviedb.org/movie/10529
https://www.themoviedb.org/movie/267806
https://www.themoviedb.org/movie/857732
https://www.themoviedb.org/movie/9889
https://www.themoviedb.org/movie/174751
https://www.themoviedb.org/movie/54318
https://www.themoviedb.org/movie/393
https://www.themoviedb.org/movie/892792
https://www.themoviedb.org/movie/563


page number 173
https://www.themoviedb.org/movie/345923
https://www.themoviedb.org/movie/340270
https://www.themoviedb.org/movie/399170
https://www.themoviedb.org/movie/754433
https://www.themoviedb.org/movie/463843
https://www.themoviedb.org/movie/434555
https://www.themoviedb.org/movie/765
https://www.themoviedb.org/movie/316023
https://www.themoviedb.org/movie/27583
https://www.themoviedb.org/movie/518755
https://www.themoviedb.org/movie/166076
https://www.themoviedb.org/movie/39145
https://www.themoviedb.org/movie/749645
https://www.themoviedb.org/movie/892977
https://www.themoviedb.org/movie/7459
https://www.themoviedb.org/movie/853
ht

https://www.themoviedb.org/movie/3563
https://www.themoviedb.org/movie/459992
https://www.themoviedb.org/movie/75736
https://www.themoviedb.org/movie/732721
https://www.themoviedb.org/movie/423988


page number 183
https://www.themoviedb.org/movie/699204
https://www.themoviedb.org/movie/1646
https://www.themoviedb.org/movie/892774
https://www.themoviedb.org/movie/297608
https://www.themoviedb.org/movie/333622
https://www.themoviedb.org/movie/9966
https://www.themoviedb.org/movie/816
https://www.themoviedb.org/movie/582570
https://www.themoviedb.org/movie/38157
https://www.themoviedb.org/movie/33217
https://www.themoviedb.org/movie/9334
https://www.themoviedb.org/movie/893936
https://www.themoviedb.org/movie/483558
https://www.themoviedb.org/movie/376866
https://www.themoviedb.org/movie/660
https://www.themoviedb.org/movie/257447
https://www.themoviedb.org/movie/16790
https://www.themoviedb.org/movie/474354
https://www.themoviedb.org/movie/21832
https://www.themoviedb.org/movie/10590






page number 193
https://www.themoviedb.org/movie/333941
https://www.themoviedb.org/movie/245913
https://www.themoviedb.org/movie/156708
https://www.themoviedb.org/movie/9678
https://www.themoviedb.org/movie/463272
https://www.themoviedb.org/movie/88005
https://www.themoviedb.org/movie/707
https://www.themoviedb.org/movie/10060
https://www.themoviedb.org/movie/270774
https://www.themoviedb.org/movie/567452
https://www.themoviedb.org/movie/258193
https://www.themoviedb.org/movie/467248
https://www.themoviedb.org/movie/376659
https://www.themoviedb.org/movie/390051
https://www.themoviedb.org/movie/439058
https://www.themoviedb.org/movie/225745
https://www.themoviedb.org/movie/241848
https://www.themoviedb.org/movie/430155
https://www.themoviedb.org/movie/203696
https://www.themoviedb.org/movie/27098


page number 194
https://www.themoviedb.org/movie/555974
https://www.themoviedb.org/movie/2057
https://www.themoviedb.org/movie/253980
https://www.themoviedb.org/movie/376290
https://www.th

In [72]:
get_data(starting=201,ending=300)

page number 201
https://www.themoviedb.org/movie/33542
https://www.themoviedb.org/movie/612972
https://www.themoviedb.org/movie/171823
https://www.themoviedb.org/movie/611698
https://www.themoviedb.org/movie/10923
https://www.themoviedb.org/movie/10552
https://www.themoviedb.org/movie/70868
https://www.themoviedb.org/movie/1581
https://www.themoviedb.org/movie/767499
https://www.themoviedb.org/movie/450806
https://www.themoviedb.org/movie/182026
https://www.themoviedb.org/movie/339404
https://www.themoviedb.org/movie/190955
https://www.themoviedb.org/movie/466622
https://www.themoviedb.org/movie/68734
https://www.themoviedb.org/movie/66125
https://www.themoviedb.org/movie/479040
https://www.themoviedb.org/movie/319075
https://www.themoviedb.org/movie/1598
https://www.themoviedb.org/movie/979


page number 202
https://www.themoviedb.org/movie/387840
https://www.themoviedb.org/movie/509730
https://www.themoviedb.org/movie/560066
https://www.themoviedb.org/movie/14113
https://www.themovie

https://www.themoviedb.org/movie/77016
https://www.themoviedb.org/movie/419831
https://www.themoviedb.org/movie/133931
https://www.themoviedb.org/movie/13683
https://www.themoviedb.org/movie/671295
https://www.themoviedb.org/movie/6637
https://www.themoviedb.org/movie/5
https://www.themoviedb.org/movie/11230
https://www.themoviedb.org/movie/1957
https://www.themoviedb.org/movie/180
https://www.themoviedb.org/movie/11152
https://www.themoviedb.org/movie/96936
https://www.themoviedb.org/movie/2662
https://www.themoviedb.org/movie/10999
https://www.themoviedb.org/movie/774714
https://www.themoviedb.org/movie/768


page number 212
https://www.themoviedb.org/movie/500916
https://www.themoviedb.org/movie/490410
https://www.themoviedb.org/movie/10495
https://www.themoviedb.org/movie/261023
https://www.themoviedb.org/movie/9477
https://www.themoviedb.org/movie/1911
https://www.themoviedb.org/movie/360605
https://www.themoviedb.org/movie/9352
https://www.themoviedb.org/movie/50393
https://www.t

https://www.themoviedb.org/movie/15357
https://www.themoviedb.org/movie/264873
https://www.themoviedb.org/movie/6404
https://www.themoviedb.org/movie/539073
https://www.themoviedb.org/movie/8487
https://www.themoviedb.org/movie/571
https://www.themoviedb.org/movie/334074
https://www.themoviedb.org/movie/45269
https://www.themoviedb.org/movie/504982
https://www.themoviedb.org/movie/21316


page number 222
https://www.themoviedb.org/movie/1443
https://www.themoviedb.org/movie/459202
https://www.themoviedb.org/movie/575479
https://www.themoviedb.org/movie/353074
https://www.themoviedb.org/movie/3635
https://www.themoviedb.org/movie/36401
https://www.themoviedb.org/movie/11678
https://www.themoviedb.org/movie/84575
https://www.themoviedb.org/movie/288905
https://www.themoviedb.org/movie/39228
https://www.themoviedb.org/movie/13008
https://www.themoviedb.org/movie/37100
https://www.themoviedb.org/movie/624620
https://www.themoviedb.org/movie/384737
https://www.themoviedb.org/movie/9946
http

https://www.themoviedb.org/movie/403052
https://www.themoviedb.org/movie/539181
https://www.themoviedb.org/movie/28635
https://www.themoviedb.org/movie/794
https://www.themoviedb.org/movie/454227


page number 232
https://www.themoviedb.org/movie/3176
https://www.themoviedb.org/movie/471014
https://www.themoviedb.org/movie/253626
https://www.themoviedb.org/movie/610394
https://www.themoviedb.org/movie/9475
https://www.themoviedb.org/movie/336560
https://www.themoviedb.org/movie/429417
https://www.themoviedb.org/movie/15159
https://www.themoviedb.org/movie/10648
https://www.themoviedb.org/movie/59962
https://www.themoviedb.org/movie/19
https://www.themoviedb.org/movie/768141
https://www.themoviedb.org/movie/6279
https://www.themoviedb.org/movie/938
https://www.themoviedb.org/movie/44037
https://www.themoviedb.org/movie/4226
https://www.themoviedb.org/movie/465003
https://www.themoviedb.org/movie/25237
https://www.themoviedb.org/movie/15544
https://www.themoviedb.org/movie/419700


page 

https://www.themoviedb.org/movie/13333
https://www.themoviedb.org/movie/416153
https://www.themoviedb.org/movie/791469
https://www.themoviedb.org/movie/10610
https://www.themoviedb.org/movie/442056
https://www.themoviedb.org/movie/318917
https://www.themoviedb.org/movie/270413
https://www.themoviedb.org/movie/84332
https://www.themoviedb.org/movie/586347
https://www.themoviedb.org/movie/206284
https://www.themoviedb.org/movie/873037
https://www.themoviedb.org/movie/9745
https://www.themoviedb.org/movie/9426
https://www.themoviedb.org/movie/69735
https://www.themoviedb.org/movie/919
https://www.themoviedb.org/movie/316152
https://www.themoviedb.org/movie/710356
https://www.themoviedb.org/movie/15657
https://www.themoviedb.org/movie/10780
https://www.themoviedb.org/movie/268092


page number 243
https://www.themoviedb.org/movie/254474
https://www.themoviedb.org/movie/13465
https://www.themoviedb.org/movie/13363
https://www.themoviedb.org/movie/38321
https://www.themoviedb.org/movie/47363

https://www.themoviedb.org/movie/41280
https://www.themoviedb.org/movie/339987
https://www.themoviedb.org/movie/39486
https://www.themoviedb.org/movie/10294
https://www.themoviedb.org/movie/61717
https://www.themoviedb.org/movie/531593
https://www.themoviedb.org/movie/214761
https://www.themoviedb.org/movie/284276
https://www.themoviedb.org/movie/10663
https://www.themoviedb.org/movie/279
https://www.themoviedb.org/movie/339380
https://www.themoviedb.org/movie/11860
https://www.themoviedb.org/movie/2026
https://www.themoviedb.org/movie/504562
https://www.themoviedb.org/movie/86829


page number 253
https://www.themoviedb.org/movie/500682
https://www.themoviedb.org/movie/486131
https://www.themoviedb.org/movie/457955
https://www.themoviedb.org/movie/527280
https://www.themoviedb.org/movie/11198
https://www.themoviedb.org/movie/397873
https://www.themoviedb.org/movie/792517
https://www.themoviedb.org/movie/9800
https://www.themoviedb.org/movie/12142
https://www.themoviedb.org/movie/50361

https://www.themoviedb.org/movie/7516
https://www.themoviedb.org/movie/377264
https://www.themoviedb.org/movie/11778
https://www.themoviedb.org/movie/9313
https://www.themoviedb.org/movie/474764
https://www.themoviedb.org/movie/186351
https://www.themoviedb.org/movie/2320
https://www.themoviedb.org/movie/8326
https://www.themoviedb.org/movie/347866
https://www.themoviedb.org/movie/74308


page number 263
https://www.themoviedb.org/movie/11092
https://www.themoviedb.org/movie/11884
https://www.themoviedb.org/movie/322240
https://www.themoviedb.org/movie/22582
https://www.themoviedb.org/movie/406759
https://www.themoviedb.org/movie/379
https://www.themoviedb.org/movie/489064
https://www.themoviedb.org/movie/626329
https://www.themoviedb.org/movie/631939
https://www.themoviedb.org/movie/375315
https://www.themoviedb.org/movie/12902
https://www.themoviedb.org/movie/70074
https://www.themoviedb.org/movie/10331
https://www.themoviedb.org/movie/150028
https://www.themoviedb.org/movie/11808
ht

https://www.themoviedb.org/movie/463116
https://www.themoviedb.org/movie/76649
https://www.themoviedb.org/movie/819998
https://www.themoviedb.org/movie/10166


page number 273
https://www.themoviedb.org/movie/9029
https://www.themoviedb.org/movie/892979
https://www.themoviedb.org/movie/681195
https://www.themoviedb.org/movie/152584
https://www.themoviedb.org/movie/4517
https://www.themoviedb.org/movie/10546
https://www.themoviedb.org/movie/300386
https://www.themoviedb.org/movie/63492
https://www.themoviedb.org/movie/227156
https://www.themoviedb.org/movie/3090
https://www.themoviedb.org/movie/12699
https://www.themoviedb.org/movie/818367
https://www.themoviedb.org/movie/11702
https://www.themoviedb.org/movie/116
https://www.themoviedb.org/movie/369883
https://www.themoviedb.org/movie/239678
https://www.themoviedb.org/movie/442495
https://www.themoviedb.org/movie/36362
https://www.themoviedb.org/movie/9604
https://www.themoviedb.org/movie/595985


page number 274
https://www.themoviedb

https://www.themoviedb.org/movie/33543
https://www.themoviedb.org/movie/2124
https://www.themoviedb.org/movie/9441
https://www.themoviedb.org/movie/446791
https://www.themoviedb.org/movie/489927
https://www.themoviedb.org/movie/328739
https://www.themoviedb.org/movie/342737
https://www.themoviedb.org/movie/17529
https://www.themoviedb.org/movie/432383
https://www.themoviedb.org/movie/11520
https://www.themoviedb.org/movie/573336
https://www.themoviedb.org/movie/560238
https://www.themoviedb.org/movie/178809
https://www.themoviedb.org/movie/62764
https://www.themoviedb.org/movie/784393
https://www.themoviedb.org/movie/134597
https://www.themoviedb.org/movie/212716
https://www.themoviedb.org/movie/14435
https://www.themoviedb.org/movie/46332
https://www.themoviedb.org/movie/24696


page number 284
https://www.themoviedb.org/movie/10586
https://www.themoviedb.org/movie/58151
https://www.themoviedb.org/movie/13828
https://www.themoviedb.org/movie/145135
https://www.themoviedb.org/movie/605

https://www.themoviedb.org/movie/1213
https://www.themoviedb.org/movie/879111
https://www.themoviedb.org/movie/451925
https://www.themoviedb.org/movie/403964
https://www.themoviedb.org/movie/15997
https://www.themoviedb.org/movie/10861
https://www.themoviedb.org/movie/298382
https://www.themoviedb.org/movie/11881
https://www.themoviedb.org/movie/753230
https://www.themoviedb.org/movie/4929
https://www.themoviedb.org/movie/9574
https://www.themoviedb.org/movie/890353
https://www.themoviedb.org/movie/317557
https://www.themoviedb.org/movie/239529
https://www.themoviedb.org/movie/549435


page number 294
https://www.themoviedb.org/movie/387844
https://www.themoviedb.org/movie/309924
https://www.themoviedb.org/movie/242095
https://www.themoviedb.org/movie/45242
https://www.themoviedb.org/movie/11878
https://www.themoviedb.org/movie/803923
https://www.themoviedb.org/movie/303991
https://www.themoviedb.org/movie/3082
https://www.themoviedb.org/movie/521647
https://www.themoviedb.org/movie/14

In [73]:
get_data(starting=301,ending=400)

page number 301
https://www.themoviedb.org/movie/10651
https://www.themoviedb.org/movie/2039
https://www.themoviedb.org/movie/13252
https://www.themoviedb.org/movie/539651
https://www.themoviedb.org/movie/874299
https://www.themoviedb.org/movie/661950
https://www.themoviedb.org/movie/376424
https://www.themoviedb.org/movie/9618
https://www.themoviedb.org/movie/11849
https://www.themoviedb.org/movie/575769
https://www.themoviedb.org/movie/301804
https://www.themoviedb.org/movie/2655
https://www.themoviedb.org/movie/536743
https://www.themoviedb.org/movie/535167
https://www.themoviedb.org/movie/74465
https://www.themoviedb.org/movie/60670
https://www.themoviedb.org/movie/17792
https://www.themoviedb.org/movie/985
https://www.themoviedb.org/movie/631060
https://www.themoviedb.org/movie/347629


page number 302
https://www.themoviedb.org/movie/10326
https://www.themoviedb.org/movie/160588
https://www.themoviedb.org/movie/81390
https://www.themoviedb.org/movie/37233
https://www.themoviedb.o

https://www.themoviedb.org/movie/3489
https://www.themoviedb.org/movie/506
https://www.themoviedb.org/movie/666390
https://www.themoviedb.org/movie/11568
https://www.themoviedb.org/movie/1441
https://www.themoviedb.org/movie/890592
https://www.themoviedb.org/movie/529485
https://www.themoviedb.org/movie/2899
https://www.themoviedb.org/movie/11797
https://www.themoviedb.org/movie/47292
https://www.themoviedb.org/movie/40377
https://www.themoviedb.org/movie/414190
https://www.themoviedb.org/movie/375107
https://www.themoviedb.org/movie/14787
https://www.themoviedb.org/movie/40508


page number 312
https://www.themoviedb.org/movie/427291
https://www.themoviedb.org/movie/10944
https://www.themoviedb.org/movie/16873
https://www.themoviedb.org/movie/322456
https://www.themoviedb.org/movie/582
https://www.themoviedb.org/movie/399905
https://www.themoviedb.org/movie/12309
https://www.themoviedb.org/movie/686487
https://www.themoviedb.org/movie/399366
https://www.themoviedb.org/movie/236399
htt

https://www.themoviedb.org/movie/333385
https://www.themoviedb.org/movie/8676
https://www.themoviedb.org/movie/37221
https://www.themoviedb.org/movie/18781
https://www.themoviedb.org/movie/12658
https://www.themoviedb.org/movie/359784
https://www.themoviedb.org/movie/97038
https://www.themoviedb.org/movie/15655
https://www.themoviedb.org/movie/73881
https://www.themoviedb.org/movie/355992


page number 322
https://www.themoviedb.org/movie/8870
https://www.themoviedb.org/movie/513736
https://www.themoviedb.org/movie/297558
https://www.themoviedb.org/movie/419704
https://www.themoviedb.org/movie/10329
https://www.themoviedb.org/movie/9268
https://www.themoviedb.org/movie/394723
https://www.themoviedb.org/movie/505379
https://www.themoviedb.org/movie/64678
https://www.themoviedb.org/movie/755130
https://www.themoviedb.org/movie/199373
https://www.themoviedb.org/movie/49517
https://www.themoviedb.org/movie/35169
https://www.themoviedb.org/movie/451480
https://www.themoviedb.org/movie/914
h

https://www.themoviedb.org/movie/287084
https://www.themoviedb.org/movie/10032
https://www.themoviedb.org/movie/437543
https://www.themoviedb.org/movie/432976


page number 332
https://www.themoviedb.org/movie/10587
https://www.themoviedb.org/movie/456750
https://www.themoviedb.org/movie/21972
https://www.themoviedb.org/movie/678703
https://www.themoviedb.org/movie/39862
https://www.themoviedb.org/movie/366141
https://www.themoviedb.org/movie/353326
https://www.themoviedb.org/movie/11687
https://www.themoviedb.org/movie/6687
https://www.themoviedb.org/movie/90369
https://www.themoviedb.org/movie/74387
https://www.themoviedb.org/movie/592695
https://www.themoviedb.org/movie/1365
https://www.themoviedb.org/movie/11228
https://www.themoviedb.org/movie/2924
https://www.themoviedb.org/movie/486947
https://www.themoviedb.org/movie/653744
https://www.themoviedb.org/movie/341012
https://www.themoviedb.org/movie/9271
https://www.themoviedb.org/movie/879986


page number 333
https://www.themovie

https://www.themoviedb.org/movie/618019
https://www.themoviedb.org/movie/287954
https://www.themoviedb.org/movie/26973
https://www.themoviedb.org/movie/491926
https://www.themoviedb.org/movie/10226
https://www.themoviedb.org/movie/453278
https://www.themoviedb.org/movie/651589
https://www.themoviedb.org/movie/548928
https://www.themoviedb.org/movie/399121
https://www.themoviedb.org/movie/795607
https://www.themoviedb.org/movie/410129
https://www.themoviedb.org/movie/9796
https://www.themoviedb.org/movie/4614
https://www.themoviedb.org/movie/183662
https://www.themoviedb.org/movie/85621
https://www.themoviedb.org/movie/152742
https://www.themoviedb.org/movie/203834
https://www.themoviedb.org/movie/13022
https://www.themoviedb.org/movie/9922


page number 343
https://www.themoviedb.org/movie/2291
https://www.themoviedb.org/movie/455108
https://www.themoviedb.org/movie/844636
https://www.themoviedb.org/movie/411632
https://www.themoviedb.org/movie/44945
https://www.themoviedb.org/movie/66

https://www.themoviedb.org/movie/10035
https://www.themoviedb.org/movie/20986
https://www.themoviedb.org/movie/445
https://www.themoviedb.org/movie/62717
https://www.themoviedb.org/movie/553141
https://www.themoviedb.org/movie/575828
https://www.themoviedb.org/movie/686245
https://www.themoviedb.org/movie/741067
https://www.themoviedb.org/movie/4538
https://www.themoviedb.org/movie/258670
https://www.themoviedb.org/movie/74849
https://www.themoviedb.org/movie/659959
https://www.themoviedb.org/movie/11571
https://www.themoviedb.org/movie/550201


page number 353
https://www.themoviedb.org/movie/23742
https://www.themoviedb.org/movie/212156
https://www.themoviedb.org/movie/392655
https://www.themoviedb.org/movie/10197
https://www.themoviedb.org/movie/115210
https://www.themoviedb.org/movie/20921
https://www.themoviedb.org/movie/859
https://www.themoviedb.org/movie/2255
https://www.themoviedb.org/movie/653601
https://www.themoviedb.org/movie/14013
https://www.themoviedb.org/movie/37414
ht

https://www.themoviedb.org/movie/9986
https://www.themoviedb.org/movie/205588
https://www.themoviedb.org/movie/134411
https://www.themoviedb.org/movie/9280
https://www.themoviedb.org/movie/10322
https://www.themoviedb.org/movie/455957
https://www.themoviedb.org/movie/10206
https://www.themoviedb.org/movie/96724
https://www.themoviedb.org/movie/9292


page number 363
https://www.themoviedb.org/movie/624372
https://www.themoviedb.org/movie/172
https://www.themoviedb.org/movie/9599
https://www.themoviedb.org/movie/423612
https://www.themoviedb.org/movie/8090
https://www.themoviedb.org/movie/407742
https://www.themoviedb.org/movie/11323
https://www.themoviedb.org/movie/584044
https://www.themoviedb.org/movie/387836
https://www.themoviedb.org/movie/212153
https://www.themoviedb.org/movie/7092
https://www.themoviedb.org/movie/866
https://www.themoviedb.org/movie/583
https://www.themoviedb.org/movie/832
https://www.themoviedb.org/movie/2112
https://www.themoviedb.org/movie/543583
https://www.

https://www.themoviedb.org/movie/13310
https://www.themoviedb.org/movie/10731
https://www.themoviedb.org/movie/513399
https://www.themoviedb.org/movie/373261


page number 373
https://www.themoviedb.org/movie/11982
https://www.themoviedb.org/movie/451500
https://www.themoviedb.org/movie/147
https://www.themoviedb.org/movie/784447
https://www.themoviedb.org/movie/538207
https://www.themoviedb.org/movie/458083
https://www.themoviedb.org/movie/476764
https://www.themoviedb.org/movie/1997
https://www.themoviedb.org/movie/14299
https://www.themoviedb.org/movie/1266
https://www.themoviedb.org/movie/279641
https://www.themoviedb.org/movie/687
https://www.themoviedb.org/movie/641662
https://www.themoviedb.org/movie/468202
https://www.themoviedb.org/movie/13312
https://www.themoviedb.org/movie/12763
https://www.themoviedb.org/movie/2295
https://www.themoviedb.org/movie/41210
https://www.themoviedb.org/movie/651070
https://www.themoviedb.org/movie/8872


page number 374
https://www.themoviedb.or

https://www.themoviedb.org/movie/269148
https://www.themoviedb.org/movie/77953
https://www.themoviedb.org/movie/4169
https://www.themoviedb.org/movie/444902
https://www.themoviedb.org/movie/592867
https://www.themoviedb.org/movie/399612
https://www.themoviedb.org/movie/526
https://www.themoviedb.org/movie/3870
https://www.themoviedb.org/movie/24548
https://www.themoviedb.org/movie/24126
https://www.themoviedb.org/movie/3053
https://www.themoviedb.org/movie/821683
https://www.themoviedb.org/movie/82767
https://www.themoviedb.org/movie/205220
https://www.themoviedb.org/movie/11066
https://www.themoviedb.org/movie/10491
https://www.themoviedb.org/movie/6961
https://www.themoviedb.org/movie/385360
https://www.themoviedb.org/movie/25538
https://www.themoviedb.org/movie/419639


page number 384
https://www.themoviedb.org/movie/9322
https://www.themoviedb.org/movie/8069
https://www.themoviedb.org/movie/293970
https://www.themoviedb.org/movie/312966
https://www.themoviedb.org/movie/210947
http

https://www.themoviedb.org/movie/604605
https://www.themoviedb.org/movie/606625
https://www.themoviedb.org/movie/507618
https://www.themoviedb.org/movie/24833
https://www.themoviedb.org/movie/13855
https://www.themoviedb.org/movie/11287
https://www.themoviedb.org/movie/16266
https://www.themoviedb.org/movie/581420
https://www.themoviedb.org/movie/209451
https://www.themoviedb.org/movie/37498
https://www.themoviedb.org/movie/891479
https://www.themoviedb.org/movie/893614
https://www.themoviedb.org/movie/82684
https://www.themoviedb.org/movie/10050
https://www.themoviedb.org/movie/9622
https://www.themoviedb.org/movie/3112


page number 394
https://www.themoviedb.org/movie/444149
https://www.themoviedb.org/movie/65496
https://www.themoviedb.org/movie/35854
https://www.themoviedb.org/movie/44629
https://www.themoviedb.org/movie/770254
https://www.themoviedb.org/movie/351901
https://www.themoviedb.org/movie/446696
https://www.themoviedb.org/movie/11282
https://www.themoviedb.org/movie/8440

In [74]:
get_data(starting=401,ending=500)

page number 401
https://www.themoviedb.org/movie/773796
https://www.themoviedb.org/movie/7508
https://www.themoviedb.org/movie/353609
https://www.themoviedb.org/movie/79694
https://www.themoviedb.org/movie/9822
https://www.themoviedb.org/movie/484747
https://www.themoviedb.org/movie/11782
https://www.themoviedb.org/movie/874166
https://www.themoviedb.org/movie/10396
https://www.themoviedb.org/movie/482936
https://www.themoviedb.org/movie/11589
https://www.themoviedb.org/movie/11848
https://www.themoviedb.org/movie/659991
https://www.themoviedb.org/movie/29437
https://www.themoviedb.org/movie/337674
https://www.themoviedb.org/movie/435707
https://www.themoviedb.org/movie/848724
https://www.themoviedb.org/movie/13368
https://www.themoviedb.org/movie/613
https://www.themoviedb.org/movie/49494


page number 402
https://www.themoviedb.org/movie/1712
https://www.themoviedb.org/movie/449749
https://www.themoviedb.org/movie/330070
https://www.themoviedb.org/movie/39229
https://www.themoviedb.o

https://www.themoviedb.org/movie/19457
https://www.themoviedb.org/movie/746463
https://www.themoviedb.org/movie/16136
https://www.themoviedb.org/movie/347158
https://www.themoviedb.org/movie/434355
https://www.themoviedb.org/movie/346671
https://www.themoviedb.org/movie/535437
https://www.themoviedb.org/movie/431562
https://www.themoviedb.org/movie/63700
https://www.themoviedb.org/movie/628446
https://www.themoviedb.org/movie/97434
https://www.themoviedb.org/movie/164457
https://www.themoviedb.org/movie/858
https://www.themoviedb.org/movie/34204
https://www.themoviedb.org/movie/372782
https://www.themoviedb.org/movie/9828


page number 412
https://www.themoviedb.org/movie/877655
https://www.themoviedb.org/movie/376166
https://www.themoviedb.org/movie/9087
https://www.themoviedb.org/movie/15417
https://www.themoviedb.org/movie/531438
https://www.themoviedb.org/movie/47166
https://www.themoviedb.org/movie/41602
https://www.themoviedb.org/movie/485337
https://www.themoviedb.org/movie/2590

https://www.themoviedb.org/movie/554581
https://www.themoviedb.org/movie/11040
https://www.themoviedb.org/movie/49014
https://www.themoviedb.org/movie/1653
https://www.themoviedb.org/movie/44321
https://www.themoviedb.org/movie/9952
https://www.themoviedb.org/movie/587429
https://www.themoviedb.org/movie/425942
https://www.themoviedb.org/movie/10986
https://www.themoviedb.org/movie/271404
https://www.themoviedb.org/movie/364223
https://www.themoviedb.org/movie/356057


page number 422
https://www.themoviedb.org/movie/315846
https://www.themoviedb.org/movie/508330
https://www.themoviedb.org/movie/269650
https://www.themoviedb.org/movie/14208
https://www.themoviedb.org/movie/415086
https://www.themoviedb.org/movie/3078
https://www.themoviedb.org/movie/1480
https://www.themoviedb.org/movie/44835
https://www.themoviedb.org/movie/13188
https://www.themoviedb.org/movie/11953
https://www.themoviedb.org/movie/8989
https://www.themoviedb.org/movie/37021
https://www.themoviedb.org/movie/513409
h

https://www.themoviedb.org/movie/401545
https://www.themoviedb.org/movie/618219
https://www.themoviedb.org/movie/103747
https://www.themoviedb.org/movie/18220
https://www.themoviedb.org/movie/17124
https://www.themoviedb.org/movie/64847
https://www.themoviedb.org/movie/79842
https://www.themoviedb.org/movie/16763


page number 432
https://www.themoviedb.org/movie/12192
https://www.themoviedb.org/movie/11817
https://www.themoviedb.org/movie/458293
https://www.themoviedb.org/movie/542921
https://www.themoviedb.org/movie/246860
https://www.themoviedb.org/movie/14854
https://www.themoviedb.org/movie/60599
https://www.themoviedb.org/movie/9296
https://www.themoviedb.org/movie/22798
https://www.themoviedb.org/movie/11431
https://www.themoviedb.org/movie/442065
https://www.themoviedb.org/movie/25750
https://www.themoviedb.org/movie/505192
https://www.themoviedb.org/movie/10907
https://www.themoviedb.org/movie/2669
https://www.themoviedb.org/movie/401
https://www.themoviedb.org/movie/313230
ht

https://www.themoviedb.org/movie/17927
https://www.themoviedb.org/movie/770315
https://www.themoviedb.org/movie/238589


page number 442
https://www.themoviedb.org/movie/10835
https://www.themoviedb.org/movie/746131
https://www.themoviedb.org/movie/609488
https://www.themoviedb.org/movie/22527
https://www.themoviedb.org/movie/11780
https://www.themoviedb.org/movie/235046
https://www.themoviedb.org/movie/614587
https://www.themoviedb.org/movie/25103
https://www.themoviedb.org/movie/1936
https://www.themoviedb.org/movie/25741
https://www.themoviedb.org/movie/19265
https://www.themoviedb.org/movie/29912
https://www.themoviedb.org/movie/438970
https://www.themoviedb.org/movie/2357
https://www.themoviedb.org/movie/617784
https://www.themoviedb.org/movie/609408
https://www.themoviedb.org/movie/453755
https://www.themoviedb.org/movie/298583
https://www.themoviedb.org/movie/483411
https://www.themoviedb.org/movie/15641


page number 443
https://www.themoviedb.org/movie/48838
https://www.themov

https://www.themoviedb.org/movie/833237
https://www.themoviedb.org/movie/53064
https://www.themoviedb.org/movie/12501
https://www.themoviedb.org/movie/412202
https://www.themoviedb.org/movie/357400
https://www.themoviedb.org/movie/14817
https://www.themoviedb.org/movie/481375
https://www.themoviedb.org/movie/4959
https://www.themoviedb.org/movie/33555
https://www.themoviedb.org/movie/9542
https://www.themoviedb.org/movie/526746
https://www.themoviedb.org/movie/94352
https://www.themoviedb.org/movie/28602
https://www.themoviedb.org/movie/505707
https://www.themoviedb.org/movie/286521
https://www.themoviedb.org/movie/71157
https://www.themoviedb.org/movie/11372
https://www.themoviedb.org/movie/15338
https://www.themoviedb.org/movie/718838


page number 453
https://www.themoviedb.org/movie/32740
https://www.themoviedb.org/movie/553608
https://www.themoviedb.org/movie/233639
https://www.themoviedb.org/movie/29154
https://www.themoviedb.org/movie/8079
https://www.themoviedb.org/movie/334531

https://www.themoviedb.org/movie/238628
https://www.themoviedb.org/movie/109099
https://www.themoviedb.org/movie/11892
https://www.themoviedb.org/movie/818274
https://www.themoviedb.org/movie/32293
https://www.themoviedb.org/movie/11327
https://www.themoviedb.org/movie/15596
https://www.themoviedb.org/movie/13666
https://www.themoviedb.org/movie/2959
https://www.themoviedb.org/movie/53487
https://www.themoviedb.org/movie/472744
https://www.themoviedb.org/movie/13934
https://www.themoviedb.org/movie/261985
https://www.themoviedb.org/movie/17915
https://www.themoviedb.org/movie/10388


page number 463
https://www.themoviedb.org/movie/9037
https://www.themoviedb.org/movie/287233
https://www.themoviedb.org/movie/323262
https://www.themoviedb.org/movie/514407
https://www.themoviedb.org/movie/606876
https://www.themoviedb.org/movie/135812
https://www.themoviedb.org/movie/388440
https://www.themoviedb.org/movie/142308
https://www.themoviedb.org/movie/14637
https://www.themoviedb.org/movie/470

https://www.themoviedb.org/movie/660637
https://www.themoviedb.org/movie/31439
https://www.themoviedb.org/movie/672647
https://www.themoviedb.org/movie/323661
https://www.themoviedb.org/movie/810873
https://www.themoviedb.org/movie/84165
https://www.themoviedb.org/movie/888155
https://www.themoviedb.org/movie/24740
https://www.themoviedb.org/movie/27586
https://www.themoviedb.org/movie/458342


page number 473
https://www.themoviedb.org/movie/253295
https://www.themoviedb.org/movie/531949
https://www.themoviedb.org/movie/20910
https://www.themoviedb.org/movie/256917
https://www.themoviedb.org/movie/240704
https://www.themoviedb.org/movie/309889
https://www.themoviedb.org/movie/79777
https://www.themoviedb.org/movie/290729
https://www.themoviedb.org/movie/1073
https://www.themoviedb.org/movie/50468
https://www.themoviedb.org/movie/37280
https://www.themoviedb.org/movie/339095
https://www.themoviedb.org/movie/483685
https://www.themoviedb.org/movie/33997
https://www.themoviedb.org/movie/

https://www.themoviedb.org/movie/17474
https://www.themoviedb.org/movie/398920
https://www.themoviedb.org/movie/48186
https://www.themoviedb.org/movie/100898
https://www.themoviedb.org/movie/15582
https://www.themoviedb.org/movie/11708


page number 483
https://www.themoviedb.org/movie/10479
https://www.themoviedb.org/movie/14885
https://www.themoviedb.org/movie/373877
https://www.themoviedb.org/movie/37632
https://www.themoviedb.org/movie/512218
https://www.themoviedb.org/movie/10503
https://www.themoviedb.org/movie/9989
https://www.themoviedb.org/movie/11176
https://www.themoviedb.org/movie/339526
https://www.themoviedb.org/movie/48231
https://www.themoviedb.org/movie/9642
https://www.themoviedb.org/movie/467
https://www.themoviedb.org/movie/463364
https://www.themoviedb.org/movie/334517
https://www.themoviedb.org/movie/260535
https://www.themoviedb.org/movie/23128
https://www.themoviedb.org/movie/571650
https://www.themoviedb.org/movie/9415
https://www.themoviedb.org/movie/20771
htt

https://www.themoviedb.org/movie/12766
https://www.themoviedb.org/movie/215211


page number 493
https://www.themoviedb.org/movie/694919
https://www.themoviedb.org/movie/463252
https://www.themoviedb.org/movie/395763
https://www.themoviedb.org/movie/505832
https://www.themoviedb.org/movie/20770
https://www.themoviedb.org/movie/20424
https://www.themoviedb.org/movie/10649
https://www.themoviedb.org/movie/401698
https://www.themoviedb.org/movie/10090
https://www.themoviedb.org/movie/632
https://www.themoviedb.org/movie/28200
https://www.themoviedb.org/movie/50601
https://www.themoviedb.org/movie/11353
https://www.themoviedb.org/movie/15004
https://www.themoviedb.org/movie/299513
https://www.themoviedb.org/movie/47599
https://www.themoviedb.org/movie/556501
https://www.themoviedb.org/movie/5146
https://www.themoviedb.org/movie/597208
https://www.themoviedb.org/movie/5552


page number 494
https://www.themoviedb.org/movie/10218
https://www.themoviedb.org/movie/518880
https://www.themoviedb

In [75]:
len(movie_data_list)

10000

In [76]:
keys = movie_data_list[0].keys()

In [77]:
keys

dict_keys(['budget(in million)', 'revenue(in million)', 'content_score', 'user_score', 'genres', 'director', 'release_year', 'release_month', 'release_day', 'release_weekday', 'run_time (in minuts)', 'title', 'url'])

In [78]:
with open('movies.csv','w',newline='') as data_input:
    data_input = csv.DictWriter(data_input,keys)
    data_input.writeheader()
    data_input.writerows(movie_data_list)

In [79]:
df = pd.read_csv('movies.csv')

In [80]:
df

,budget(in million),revenue(in million),content_score,user_score,genres,director,release_year,release_month,release_day,release_weekday,run_time (in minuts),title,url
0,150.000,430.238384,100,79.0,"Action,Adventure,Fantasy",Destin Daniel Cretton,2021,9,3,4,132.0,Shang-Chi and the Legend of the Ten Rings,https://www.themoviedb.org/movie/566525
1,242.000,708.000000,100,76.0,"Adventure,Action,Thriller",Cary Joji Fukunaga,2021,9,30,3,163.0,No Time to Die,https://www.themoviedb.org/movie/370172
2,110.000,424.000000,100,68.0,"Science Fiction,Action,Adventure",Kelly Marcel,2021,10,1,4,97.0,Venom: Let There Be Carnage,https://www.themoviedb.org/movie/580489
3,NaN,NaN,100,82.0,"Science Fiction,Drama,Adventure",Miguel Sapochnik,2021,11,4,3,115.0,Finch,https://www.themoviedb.org/movie/522402
4,NaN,NaN,100,56.0,"Action,Thriller,Science Fiction",Edward Drake,2021,12,16,3,108.0,Apex,https://www.themoviedb.org/movie/763164
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0.475,NaN,100,53.0,"Thriller,Crime",John Barr,2020,5,15,4,89.0,Blood and Money,https://www.themoviedb.org/movie/691812
9996,35.000,13.678913,86,61.0,"Action,Western",Les Mayfield,2001,8,17,4,94.0,American Outlaws,https://www.themoviedb.org/movie/13496
9997,30.000,34.252847,100,58.0,Comedy,Richard Linklater,2005,7,22,4,113.0,Bad News Bears,https://www.themoviedb.org/movie/13341
9998,NaN,NaN,100,72.0,"Drama,Comedy,Romance",Ernst Marischka,1956,12,1,5,107.0,Sissi: The Young Empress,https://www.themoviedb.org/movie/458
